In [3]:
import pandas as pd
import numpy as np
import random
import time

# Reading Data

In [37]:
df=pd.read_excel("Data/TimeTable, FSC, Spring-2023.xlsx",sheet_name=None)["Monday"]
df1=pd.read_csv("Data/Courses Allocation, FSC, NUCES, Islamabad, Spring-2023 - Computing-Theory.csv")[["Course","Section","Course Instructor"]]
df2=pd.read_csv("Data/Courses Allocation, FSC, NUCES, Islamabad, Spring-2023 - MG.csv")[["Course","Section","Course Instructor"]]
df3=pd.read_csv("Data/Courses Allocation, FSC, NUCES, Islamabad, Spring-2023 - S&H.csv")[["Course","Section","Course Instructor"]]
rooms=df["Monday"][4:-17].tolist()
timeslots=["08:30-09:50","10:00-11:20","11:30-12:50","01:00-02:20","02:30-03:50","03:55-05:15"]
df1=df1.ffill(axis=0)
df2=df2.ffill(axis=0)
df3=df3.ffill(axis=0)
df1=df1.append(df2)
df1=df1.append(df3)
df1=df1.dropna()
alleles=df1.values.tolist()

C:\Users\Hamza\AppData\Local\Temp\ipykernel_12016\4045390096.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1=df1.append(df2)
C:\Users\Hamza\AppData\Local\Temp\ipykernel_12016\4045390096.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1=df1.append(df3)


In [5]:
len(alleles[369])

3

# Genetic Algorithm
### My fitness function calculates total number of invalidies.
### If teachers name appears in column (or timetable slot) twice it is counted as 1 invalidity. If thrice then 2 invalidities and so on.
### If a section name appears in column (or timetable slot) twice it is counted as 1 invalidity. If thrice then 2 invalidities and so on.
### If teachers name appears in whole timetable 4 times it is counted as 1 invalidity. If it appears 5 times then 2 invalidities and so on.
### If sections name appears in whole timetable 4 times it is counted as 1 invalidity. If it appears 5 times then 2 invalidities and so on.
### All of the invalidities are combined to form fitness value
### Lower the fitness value the better
### At end, 0 fitness value will indicate all invalidites have been removed!

In [50]:
def generate_population(n):
    pop=[]
    for k in range(0,n):
        sample=random.sample(range(0,370),210)
        index=0
        arr=np.empty((len(rooms),6), dtype=object)
        for i in range(0,len(rooms)):
            for j in range(0,6):
                arr[i][j]=alleles[sample[index]]
                index+=1
        pop.append(arr)
    return pop

def fitnessFunction(population,iteration,population_size):
    fitness=[]
    for k in range(0,len(population)):
        fit_teacher=0
        fit_student=0
        for i in range(0,6):
            teacher={}
            student={}
            for j in range(0,len(rooms)):
                if population[k][j][i][2] in teacher.keys():
                    fit_teacher+=1
                else:
                    teacher[population[k][j][i][2]]=0

                if population[k][j][i][1] in student.keys():
                    fit_student+=1
                else:
                    student[population[k][j][i][1]]=0

        teacher2={}
        student2={}
        for i in range(0,len(rooms)):
            for j in range(0,6):
                if population[k][i][j][2] in teacher2.keys():
                    teacher2[population[k][i][j][2]]+=1
                    if teacher2[population[k][i][j][2]]>3:
                        fit_teacher+=1
                else:
                    teacher2[population[k][i][j][2]]=1

                if population[k][i][j][1] in student2.keys():
                    student2[population[k][i][j][1]]+=1
                    if student2[population[k][i][j][1]] > 3:
                        fit_student+=1
                else:
                    student2[population[k][i][j][1]]=1

        fitness.append(fit_teacher+fit_student)

    bestFitness=min(fitness)
    indx=fitness.index(bestFitness)
    bestChromosome=population[indx]
    print(fitness)
    for i in range(0,population_size):
        bad=fitness.index(max(fitness))
        del population[bad]
        del fitness[bad]

    if iteration%100000==0:
        print(fitness)
    print(fitness)
    return population,bestChromosome,bestFitness

def selection(parents):
    return random.sample(parents,2)

def crossover(parents):
    cut=random.randint(1,34)
    child1=np.vstack((parents[0][:cut,:],parents[1][cut:,:]))
    child2=np.vstack((parents[1][:cut,:],parents[0][:cut,:]))
    return child1,child2

def mutation(child1,iteration,m_ratio):
    if (iteration*m_ratio)%5==0:
        child1[random.randint(0,34)][random.randint(0,5)]=alleles[random.randint(0,369)]
    return child1

In [51]:
iteration=0
population_size=20
population=generate_population(population_size) #Generate 10 chromosomes
while True:
    for i in range(0,len(population)//2):
        parents=selection(population)
        child1,child2=crossover(parents)
        child1=mutation(child1,iteration,0.1)
        child2=mutation(child1,iteration,0.1)
        population.append(child1)
        population.append(child2)

    population,bestChromosome,bestFitness=fitnessFunction(population,iteration,population_size)
    if bestFitness == 0 :
        print("Clash-Free TimeTable created!")
        print(bestChromosome)
        break
    iteration+=1

[82, 91, 80, 84, 77, 108, 74, 90, 79, 78, 84, 85, 82, 81, 89, 81, 93, 83, 88, 90, 96, 96, 107, 107, 112, 112, 96, 96, 104, 104, 108, 108, 107, 107, 92, 92, 120, 120, 118, 118]
[82, 91, 80, 84, 77, 74, 90, 79, 78, 84, 85, 82, 81, 89, 81, 83, 88, 90, 92, 92]
[82, 91, 80, 84, 77, 74, 90, 79, 78, 84, 85, 82, 81, 89, 81, 83, 88, 90, 92, 92]
[82, 91, 80, 84, 77, 74, 90, 79, 78, 84, 85, 82, 81, 89, 81, 83, 88, 90, 92, 92, 90, 90, 100, 100, 115, 115, 88, 88, 97, 97, 107, 107, 106, 106, 92, 92, 128, 128, 114, 114]
[82, 80, 84, 77, 74, 79, 78, 84, 85, 82, 81, 89, 81, 83, 88, 90, 90, 90, 88, 88]
[82, 80, 84, 77, 74, 79, 78, 84, 85, 82, 81, 89, 81, 83, 88, 90, 90, 90, 88, 88, 107, 107, 111, 111, 112, 112, 82, 82, 104, 104, 114, 114, 104, 104, 88, 88, 92, 92, 94, 94]
[82, 80, 84, 77, 74, 79, 78, 84, 85, 82, 81, 81, 83, 88, 88, 88, 82, 82, 88, 88]
[82, 80, 84, 77, 74, 79, 78, 84, 85, 82, 81, 81, 83, 88, 88, 88, 82, 82, 88, 88, 110, 110, 94, 94, 97, 97, 93, 93, 113, 113, 119, 119, 102, 102, 97, 97, 9

# Final TimeTable with no clashes and not more than 3 classes for any student/Teacher

In [52]:
df=pd.DataFrame(bestChromosome,columns=timeslots)
df.insert(loc=0,column='Rooms',value=rooms)
df.to_csv("Output.csv")
df

,Rooms,08:30-09:50,10:00-11:20,11:30-12:50,01:00-02:20,02:30-03:50,03:55-05:15
0,C-301,"[Operating Systems (CS), CS-4A/Y, Dr. Adnan Ta...","[Statistical Modelling, CS-6E, Mr. Muhammad Al...","[Machine Learning (BS-AI), AI-A, Dr. Uzair Iqbal]","[Database Systems (SE), SE-4Q, Dr. Javeria Imt...","[Web Engineering (SE), SE-6S, Mr. Irfan Ullah]","[Software Engineering (CS), CS-6C, Dr. Ejaz Ah..."
1,C-302,"[Digital Logic Design (CS), CS-2F, Dr. Niaz Ah...","[Professional Practices in IT (AI), AI-8J, Dr....","[Communication and Presentation Skills (CS), C...","[Parallel and Distributed Computing (CS), CS-6...","[Probability and Statistics (CS), CS-4H, Sana ...","[Data Structures (All Programs), D, Mr. Bilal ..."
2,C-303,"[Entrepreneurship (DS), DS-N, Ayesha Yaqoob]",[Calculus and Analytical Geometry (All Program...,"[Object Oriented Programming (CY), CY-2A, Mr. ...","[Data Structures (All Programs), C, Ms. Amna I...",[Computer Organization and Assembly Language (...,"[Programming Fundamentals (All Programs), E, M..."
3,C-304,"[Artificial Intelligence (DS), DS-6N, Mr. Umai...","[Object Oriented Programming (CS), CS-Z, Dr. J...","[Differential Equations (AI), AI-2B, Sehrish H...","[Parallel and Distributed Computing (AI), AI-6...","[Communication and Presentation Skills (CS), C...","[Discrete Structures (CY), CY-2D, Dr. Subhan U..."
4,C-305,"[Probability and Statistics (CS), CS-4B, Khali...","[Database Systems (CS), CS-4E/V, Dr. Amna Bash...","[Object Oriented Programming (CS), CS-2A, Mr. ...","[Operating Systems (CS), CS-4B/Z, Dr. Faisal C...","[Statistical Modelling, CS-6D, Dr. Shahzad Sal...","[Communication and Presentation Skills (AI), A..."
5,C-307,"[Object Oriented Programming (SE), SE-2G, Ms. ...","[Communication and Presentation Skills (CY), C...","[Communication and Presentation Skills (SE), ...","[Discrete Structures (CY), CY-2A, Ms. Sobia Ra...","[Final Year Project-I (DS), DS-N, FYP Committee]","[Computer Networks (CY), CY-4T, Dr. Muhammad A..."
6,C-308,"[Theory of Automata, CS-A, Ms. Tajwar Mehmood]","[Differential Equations (CS), CS-2D, Arif Huss...","[Applied Physics, CS-B, Aisha Ijaz]","[Statistical Modelling, CS-6C, Mr. Muhammad Al...","[Database Systems (CS), CS-4C/D, Ms. Noor ul Ain]","[Discrete Structures (SE), SE-2A, Ms. Shafaq R..."
7,C-309,"[Communication and Presentation Skills (SE), ...","[Communication and Presentation Skills (SE), ...","[Pakistan Studies (CS), CS-2G, Gul-e-Aisha]",[Applied Artificial Intelligence (SE Elective ...,"[Web Programming, CS-A, Ms. Sidra Khalid]","[Islamic Studies/Ethics (SE), SE-2D, Sobia She..."
8,C-310,"[Differential Equations (DS), DS-2C, Sumaira A...","[Islamic Studies/Ethics (AI), AI-2C, Muhammad ...","[MLOps (AI), AI-8K, Mr. Samiullah]","[Communication and Presentation Skills (AI), A...","[Islamic Studies/Ethics (AI), AI-2C, Muhammad ...","[Statistical Modelling, CS-6F, Dr. Mukhtar Ull..."
9,C-311,"[Advanced Statistics (DS), DS-4N, Dr. Ahmed Din]","[Islamic Studies/Ethics (DS), DS-2B, Yahya Ma...","[Database Systems (CS), CS-4B/Z, Dr. Shujaat H...","[Object Oriented Programming (CS), CS-2B, Mr. ...","[Parallel and Distributed Computing (DS), DS-6...",[Computer Organization and Assembly Language (...
